In [17]:
# install.packages('rgdal')

In [23]:
library(data.table)
library(sf)
# remove.packages( 'terra')
# install.packages('rgdal')
library(magrittr)
library(ggplot2)
library(units)
library(stringr)
library(moments)
library(fasterize)
library(raster)
library(exactextractr)
library(EnvStats)
library(distributionsrd)
library(nngeo)
library(parallel)
library(spex)

In [29]:
conn <- RPostgreSQL::dbConnect("PostgreSQL",
                            host = 'mild-williams-pear.db.elephantsql.com',
                            dbname = "jifesypi",
                            user = "jifesypi",
                            password ="Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr")

In [28]:
RPostgreSQL::dbDisconnect(conn)

In [5]:
parcelnumber <- '073-260-056'

In [ ]:
parcel_dt <- st_read('/work/sonoma.gpkg', query = "SELECT * FROM sonoma WHERE parcelnumb = '073-260-056'") %>% data.table

In [ ]:
b_dist <- 500

In [ ]:
conn <- RPostgreSQL::dbConnect("PostgreSQL",
                            host = 'mild-williams-pear.db.elephantsql.com',
                            dbname = "jifesypi",
                            user = "jifesypi",
                            password ="Qxds23zjkpIIU343-GGHNlxqFdD3Pdlr")
dist_list <- b_dist
b_dist <- max(b_dist)

query_wkt <- st_as_text(parcel_dt$geom , EWKT = TRUE)
query_sql <- paste0("SELECT * FROM public.california_structures WHERE ST_Intersects(geom,'",query_wkt,"')")
parcel_structures_dt <- st_read(conn,query=query_sql) %>% data.table

query_wkt <- st_as_text(parcel_dt$geom %>% st_transform(6414) %>% st_buffer(b_dist)%>% st_transform(4326), EWKT = TRUE)
query_sql <- paste0("SELECT * FROM public.california_parcels WHERE ST_Intersects(geom,'",query_wkt,"')")
neighborhood_dt <- st_read(conn,query=query_sql) %>% data.table

query_wkt <- st_as_text(neighborhood_dt$geom %>% st_as_sf %>% st_bbox %>% st_as_sfc  %>%  st_transform(6414) %>% st_transform(4326), EWKT = TRUE)
query_sql <- paste0("SELECT * FROM public.california_structures WHERE ST_Intersects(geom,'",query_wkt,"')")
neighborhood_structures_dt <- st_read(conn,query=query_sql) %>% data.table
RPostgreSQL::dbDisconnect(conn)

In [8]:
assembly_dt <-  fread('/work/asm_pareto_dt.csv')
vegetation_dt <-  fread('/work/veg_pareto_dt.csv')
structural_dt <-  fread('/work/bld_pareto_dt.csv')

In [ ]:
biomass_rast <- raster('/work/Stack_Sonoma_2021_biomass.tif')
vegetation_rast <- raster('/work/Stack_Sonoma_2021_classification.tif')

In [ ]:
samp_biomass_rast <-    crop(biomass_rast,parcel_dt$geom %>% st_transform(6414) %>% st_buffer(30) %>% st_transform(4326)   %>% as('Spatial'))
values(samp_biomass_rast)[values(samp_biomass_rast)>1000] <- NA
samp_vegetation_rast   <-  crop(vegetation_rast,parcel_dt$geom %>% st_transform(6414) %>% st_buffer(30) %>% st_transform(4326)   %>% as('Spatial')) 

In [13]:
writeRaster(samp_biomass_rast,'/work/processing_folder/parcel_biomass.tif', overwrite=TRUE)
writeRaster(samp_vegetation_rast,'/work/processing_folder/parcel_class.tif', overwrite=TRUE)

In [ ]:
# system("gdal_polygonize.py '/work/processing_folder/parcel_class.tif' '/work/processing_folder/parcel_class.shp'")

In [21]:
samp_vegetation_sf <- samp_vegetation_rast %>% spex::polygonize() %>% st_as_sf %>% st_intersection(.,parcel_dt$geom) %>% st_as_sf %>% data.table
samp_vegetation_sf$class <- NA
samp_vegetation_sf$class[samp_vegetation_sf$lc %in% c(0,4,2,7)] <- 'Grass'
samp_vegetation_sf$class[samp_vegetation_sf$lc %in% c(5,11,13,14)] <- 'Shrub'
samp_vegetation_sf$class[samp_vegetation_sf$lc %in% c(1,3,6,8,9,10,12)] <- 'Tree'
samp_vegetation_sf$class[samp_vegetation_sf$lc %in% c(15)] <- 'Water'

In [27]:
samp_vegetation_sf$class <- NA
samp_vegetation_sf$class[samp_vegetation_sf$Stack_Sonoma_2021_classification %in% c(0,4,2,7)] <- 'Grass'
samp_vegetation_sf$class[samp_vegetation_sf$Stack_Sonoma_2021_classification %in% c(5,11,13,14)] <- 'Shrub'
samp_vegetation_sf$class[samp_vegetation_sf$Stack_Sonoma_2021_classification %in% c(1,3,6,8,9,10,12)] <- 'Tree'
samp_vegetation_sf$class[samp_vegetation_sf$Stack_Sonoma_2021_classification %in% c(15)] <- 'Water'

In [32]:
samp_vegetation_sf$area <- st_area(samp_vegetation_sf$geometry)
samp_vegetation_sf$MGHa <- exact_extract(samp_biomass_rast,samp_vegetation_sf$geometry,weights='area',fun='weighted_mean')
samp_vegetation_sf$MG <- samp_vegetation_sf$MGHa*(samp_vegetation_sf$area %>% as.numeric)*0.0001

In [48]:
samp_structure_sf<- samp_vegetation_sf[1,]
samp_structure_sf$Stack_Sonoma_2021_classification <- NA
samp_structure_sf$MGHa <- NA
samp_structure_sf$MG <- NA
samp_structure_sf$class <- 'Structure'
samp_structure_sf <- samp_structure_sf[rep(1,length(parcel_structures_dt$geom)),]
samp_structure_sf$geometry <- parcel_structures_dt$geom
samp_structure_sf <- samp_structure_sf %>% st_as_sf %>% st_cast('POLYGON') %>% data.table
samp_structure_sf$area <- st_area(samp_structure_sf$geometry)

In [59]:
samp_structure_sf <- samp_structure_sf[,names(samp_vegetation_sf),with=F]

In [ ]:
samp_vegetation_sf$geometry <- samp_vegetation_sf$geometry %>% st_collection_extract("POLYGON")

In [71]:
samp_structure_sf$geometry <- samp_structure_sf$geometry %>% st_collection_extract("POLYGON")

In [72]:
samp_structure_sf %>% head

samp_vegetation_sf <- samp_vegetation_sf  %>% st_as_sf %>% st_cast('POLYGON') %>% data.table

In [43]:
samp_ember_sf <- rbind(samp_vegetation_sf,samp_structure_sf)
samp_ember_sf$class %>% table

In [76]:
samp_ember_sf[,area:=st_area(geometry)]

In [95]:
k<-10
dim <- 10

In [101]:
samp_ember_sf$area %>% which.max

In [42]:
samp_ember_sf$geometry %>% st_crs

In [80]:
samp_ember_sf <- samp_ember_sf %>% st_as_sf %>% st_collection_extract('POLYGON')

In [102]:
class_out <- 'Tree'
samp_ember_dt[samp_ember_dt$class == class_out]
# [51,] 
# %>% st_as_sf %>% st_area()

In [ ]:
class_out <-'Tree'
samp_ember_dt <- samp_ember_sf %>% data.table
51 %>% lapply(function(x){ember_rast_list(x,samp_ember_dt[class == class_out],class_out = class_out)})

In [114]:
samp_ember_sf[samp_ember_sf[['class']] == class_out,] %>% nrow

In [ ]:
class_out <- 'Tree'
fold <- '/work/processing_folder/r_test/' %>% paste0(.,class_out)
# '/work/processing_folder/r_test/' %>% paste0(.,class_out)
fold %>% list.files( include.dirs = F, full.names = T, recursive = T) %>% file.remove()
outlist_<- 1:nrow(samp_ember_sf[samp_ember_sf[['class']] == class_out,]) %>% lapply(function(x){ember_rast_list(x,samp_ember_sf[samp_ember_sf[['class']] == class_out,],class_out = class_out)})

In [49]:
outlist_ <- fold %>% list.files( include.dirs = F, full.names = T, recursive = T) %>% lapply(raster)
outlist_$fun <- sum
outlist_rast <- outlist_ %>% do.call(mosaic,.)
writeRaster(outlist_rast,'test.tif',overwrite=T)

In [ ]:
row = 1
class_out <- 'Water'

In [94]:
# samp_ember_sf %>% names

In [111]:
ember_rast_list <- function(row, samp_ember_sf = samp_ember_sf %>% data.table,class_out = class_out){
    # tryCatch({
    print(row)
    # print(samp_ember_sf[row  ,])

    x <-   samp_ember_sf[row  ,] %>% st_as_sf %>% st_transform("EPSG:6414")
    x$area <- st_area(x$geometry)
    embers<-100
    param_dt <- assembly_dt[ `Wall Type` == 'A'  &`Wind Speed` %in% 'High'&Material=='OSB_FC'&`Roof (Sheathing)`=='NCS',]
    coef <- 1 
    if(x$class %in% c('Structure')){
    param_dt <- assembly_dt[ `Wall Type` == 'A'  &`Wind Speed` %in% 'High'&Material=='OSB_FC'&`Roof (Sheathing)`=='NCS',]
    embers <- (param_dt$n*st_area(x %>%st_as_sf) %>% as.numeric)*6
    coef <- 1 
    }
    if(x$class %in% c('Tree','Shrub','Grass')){
    param_dt <- vegetation_dt[`Vegetation Type` == x$class &`Wind Speed` %in% 'High']
    embers <- (x$MG*1000/5)*param_dt$n
    coef <- 1
    }
    if(x$class %in% c('Water')){
    # param_dt <- vegetation_dt[`Vegetation Type` == x$class &`Wind Speed` %in% 'High']
    embers <- 100
    coef <- 1
    }
    if(x$class %in% c('Shrub','Grass')){embers <- ((x$area)*param_dt$n/2) %>% as.numeric}
    if((embers%>% as.numeric) >10000000){coef <- embers/10000000 %>% as.numeric;embers <- 10000000}
    embers <- max(embers,1,na.rm=T)
    pareto_dt <- data.table(p=rpareto(embers, k = param_dt$k_dist_m, xmin = param_dt$xmin_dist_m))
    pareto_dt[,x:=ceiling(p/k)*k]
    pareto_dt<- pareto_dt[order(x),list(l = length(p)),by=x]
    pareto_dt[,l:=l*coef]
    pareto_dt[,d:=l/sum(l)]
    pareto_dt[,c:=cumsum(d)]
    pareto_dt[,area:=((x^2)*pi -((x-k)^2)*pi )]
    pareto_dt[,la:=l/area]
    pareto_dt <- pareto_dt[la>40& !is.infinite(la),]

    buffered_sf <- c(0,pareto_dt$x)  %>% lapply(function(y){st_buffer(x$geometry,y) %>% st_as_sf}) %>% do.call(rbind,.)

    buffered_sf$d <- c(1,pareto_dt$d)
    buffered_sf$l <-  c(pareto_dt$l[1],pareto_dt$l)
    buffered_sf$la <-  c(pareto_dt$la[1],pareto_dt$la)
    buffered_sf$area <- st_area(buffered_sf)
    buffered_sf$area_adj <- c(0,buffered_sf$area[1:(nrow(buffered_sf)-1)])[1:nrow(buffered_sf)]
    buffered_sf$area_ring <- (buffered_sf$area %>% as.numeric) - (buffered_sf$area_adj %>% as.numeric)

    buffered_sf$la_2 <- (buffered_sf$d*embers)/buffered_sf$area_ring


    buffered_sf <- buffered_sf[order(buffered_sf$d)  ,]
    buffered_rast <-  parcel_dt %>% st_as_sf %>%st_transform("EPSG:6414") %>% st_buffer(500)   %>% raster(.,resolution = c(dim,dim)) %>% projectRaster(.,crs="EPSG:4326")
    out_rast <- fasterize(buffered_sf %>% st_transform("EPSG:4326"),buffered_rast,'la_2',fun = 'sum',background = 0)

    writeRaster(out_rast,'/work/processing_folder/r_test/' %>% paste0(.,class_out,'/',row,'.tif'),overwrite=T)

    }

In [ ]:
buffered_sf %>% st_write('test4.gpkg')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5eceab32-5823-49b2-9c95-72828e51fba3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>